## Loading training & test datasets

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('image_emotion3.csv')
df

,filename,emotion
0,image_06637.jpg,angry
1,image_06638.jpg,angry
2,image_06639.jpg,disgust
3,image_06641.jpg,neutral
4,image_06642.jpg,sad
...,...,...
5831,image_08036.jpg,angry
5832,image_08035.jpg,neutral
5833,image_08038.jpg,sad
5834,image_08028.jpg,fear


In [3]:
def categorize(row):
    if row['emotion'] == 'angry':
        return 1
    elif row['emotion'] == 'sad':
        return 2
    elif row['emotion'] == 'happy':
        return 3
    elif row['emotion'] == 'fear':
        return 4
    elif row['emotion'] == 'surprise':
        return 5
    elif row['emotion'] == 'disgust':
        return 6
    else:
        return 7

In [4]:
df['label'] = df.apply(lambda x: categorize(x), axis=1)
df

,filename,emotion,label
0,image_06637.jpg,angry,1
1,image_06638.jpg,angry,1
2,image_06639.jpg,disgust,6
3,image_06641.jpg,neutral,7
4,image_06642.jpg,sad,2
...,...,...,...
5831,image_08036.jpg,angry,1
5832,image_08035.jpg,neutral,7
5833,image_08038.jpg,sad,2
5834,image_08028.jpg,fear,4


In [5]:
df.to_csv('train_emotions2.csv', index=False)

In [6]:
df_test = pd.read_csv('cropped_images_labelled.csv')
df_test

,filename,emotion
0,cropped_images/img_1.jpg,Neutral
1,cropped_images/img_2.jpg,Neutral
2,cropped_images/img_3.jpg,Neutral
3,cropped_images/img_4.jpg,Neutral
4,cropped_images/img_5.jpg,Angry
...,...,...
761,cropped_images/img_762.jpg,Sad
762,cropped_images/img_763.jpg,Neutral
763,cropped_images/img_764.jpg,Neutral
764,cropped_images/img_765.jpg,Happy


In [7]:
def categorize_test(row):
    if row['emotion'] == 'Angry':
        return 1
    elif row['emotion'] == 'Sad':
        return 2
    elif row['emotion'] == 'Happy':
        return 3
    elif row['emotion'] == 'Fear':
        return 4
    elif row['emotion'] == 'Surprise':
        return 5
    elif row['emotion'] == 'Disgust':
        return 6
    else:
        return 7

In [8]:
df_test['label'] = df_test.apply(lambda x: categorize_test(x), axis=1)
df_test

,filename,emotion,label
0,cropped_images/img_1.jpg,Neutral,7
1,cropped_images/img_2.jpg,Neutral,7
2,cropped_images/img_3.jpg,Neutral,7
3,cropped_images/img_4.jpg,Neutral,7
4,cropped_images/img_5.jpg,Angry,1
...,...,...,...
761,cropped_images/img_762.jpg,Sad,7
762,cropped_images/img_763.jpg,Neutral,7
763,cropped_images/img_764.jpg,Neutral,7
764,cropped_images/img_765.jpg,Happy,3


## Setup

### Import packages

In [9]:
import cv2
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torchvision.io import read_image
from torchvision.transforms import ToTensor, ToPILImage, Normalize, Compose

### Set device

In [10]:
device = None

if device == None:
    if torch.cuda.is_available():
        print("CUDA used")
        device = torch.device('cuda')

        # Should be > 0
        print(torch.cuda.device_count())

        # Index of device used (can be 0)
        print(torch.cuda.current_device())

        # GPU location
        print(torch.cuda.device(0))

        # Name of GPU
        print(torch.cuda.get_device_name(0))

    else:
        print("CPU used")
        device = torch.device('cpu')

# Force CPU use
# device = torch.device('cpu')

CPU used


### Load data

In [11]:
training_data = []

for i in range(0, len(df)):
    image, label = df.iloc[i]['filename'], df.iloc[i]['label']
    path = 'portrait_faces/' + image
    new_row = [path, int(label)]
    training_data.append(new_row)

In [12]:
print(len(training_data))

5836


In [13]:
train_set = training_data[0:int(len(training_data) * 0.8)]
validation_set = training_data[int(len(training_data) * 0.8):]
print(len(train_set) + len(validation_set))

5836


In [14]:
test_set_filename = []
test_set = []

for i in range(0, len(df_test)):
    image, label = df_test.iloc[i]['filename'], df_test.iloc[i]['label']
    new_row = [image, int(label)]
    test_set_filename.append(image)
    test_set.append(new_row)

In [15]:
print(len(test_set))

766


In [16]:
# Create a map for the labels and their codes
labels = ['angry', 'sad', 'happy', 'fear', 'surprise', 'disgust', 'neutral']
class_map = {}
i = 1
    
for label in labels:
    class_map[i] = label
    i += 1

In [17]:
print(class_map)

{1: 'angry', 2: 'sad', 3: 'happy', 4: 'fear', 5: 'surprise', 6: 'disgust', 7: 'neutral'}


In [18]:
# Class for loading of the data
class own_data(Dataset):
    def __init__(self, train=0, transform=None):
        if train == 0:
            self.data = train_set
        elif train == 1:
            self.data = validation_set
        else:
            self.data = test_set
            
        self.transform = transform
        self.to_pil = ToPILImage()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_id = self.data[idx]
        image = read_image(img_path)
        
        if self.transform:
            image = self.transform(self.to_pil(image))
        
        return image, class_id

## Model

### Initializing VGG-19 pretrained model

In [19]:
weights = models.VGG19_Weights.DEFAULT
preprocess = weights.transforms()
vgg19 = models.vgg19(weights=weights)
print(vgg19)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [21]:
# Freeze model parameters
for param in vgg19.parameters():
    param.requires_grad = False
    
# Change the final layer of ResNet50 Model for Transfer Learning
vgg19.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096), 
    nn.ReLU(inplace=True), 
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=4096, bias=True), 
    nn.ReLU(inplace=True), 
    nn.Dropout(p=0.5, inplace=False), 
    nn.Linear(in_features=4096, out_features=7, bias=True)
)

print(vgg19)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [39]:
# Define the transform function
# test_transform = transforms.Compose([
#     transforms.Resize(232), 
#     transforms.ToTensor(), 
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

# train_transform = transforms.Compose([
#     transforms.Resize(256), 
#     transforms.CenterCrop(size=224), 
#     transforms.ToTensor(), 
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

In [22]:
# Initialize the batch size
batch_size = 64

In [23]:
# Load the train and test data
train_data = own_data(0, preprocess)
validation_data = own_data(1, preprocess)
test_data = own_data(2, preprocess)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

### Set model to train

In [24]:
model = vgg19
vgg19.train()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [25]:
## Some loss functions
nll = nn.NLLLoss()

cross = nn.CrossEntropyLoss()

kldiv = nn.KLDivLoss(reduction="batchmean")

In [26]:
# Some optimizers
sgd = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

adam = optim.Adam(model.parameters(), lr=3e-4)

RMSProp = optim.RMSprop(model.parameters(), lr=1e-3, momentum=0.9)

adamw = optim.AdamW(model.parameters(), lr=3e-4)

### Define training & test functions for model

In [27]:
# Define the train model function
def train_model(train_loader, model, loss_function, optimizer, batch_size, device=None):
    for batch, (image, label) in enumerate(train_loader):
        # Move labels and images to GPU
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        output = model(image)
        label = label - 1
        loss_func = loss_function(output, label)
        loss_func.backward()
        optimizer.step()
        
        if batch % 64 == 0:
            print('[%5d] loss: %.3f' % (batch + 1, loss_func.item()))

In [28]:
# Test function
def validate_model(validation_loader, model, loss_function, batch_size, device=None):
    size = len(validation_loader.dataset)
    validation_loss, correct = 0, 0
    
    with torch.no_grad():
        for image, label in validation_loader:
            # Move labels and images to GPU
            image = image.to(device)
            label = label.to(device)
            
            output = model(image)
            label = label - 1
            validation_loss += loss_function(output, label).item()
            correct += (output.argmax(1) == label).type(torch.float).sum().item()
    
    validation_loss /= batch_size
    correct /= size
    print(f"Test error: \n Accuracy: {(100 * correct):>0.01f}%, Avg loss: {validation_loss:>8f} \n")

### Run model for epochs

In [29]:
epochs = 7
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model.train()
    train_model(train_loader, model, cross, sgd, batch_size, device=device)
    model.eval()
    validate_model(validation_loader, model, cross, batch_size, device=device)
print("Done!")

Epoch 1
-------------------------------
[    1] loss: 1.966
[   65] loss: 1.344
Test error: 
 Accuracy: 59.6%, Avg loss: 0.353936 

Epoch 2
-------------------------------
[    1] loss: 1.231
[   65] loss: 1.079
Test error: 
 Accuracy: 65.9%, Avg loss: 0.271151 

Epoch 3
-------------------------------
[    1] loss: 1.076
[   65] loss: 0.792
Test error: 
 Accuracy: 68.7%, Avg loss: 0.252806 

Epoch 4
-------------------------------
[    1] loss: 0.731
[   65] loss: 0.673
Test error: 
 Accuracy: 70.1%, Avg loss: 0.250457 

Epoch 5
-------------------------------
[    1] loss: 0.604
[   65] loss: 0.582
Test error: 
 Accuracy: 69.3%, Avg loss: 0.235559 

Epoch 6
-------------------------------
[    1] loss: 0.621
[   65] loss: 0.685
Test error: 
 Accuracy: 71.6%, Avg loss: 0.242843 

Epoch 7
-------------------------------
[    1] loss: 0.599
[   65] loss: 0.606
Test error: 
 Accuracy: 70.7%, Avg loss: 0.220785 

Done!


In [36]:
torch.save(model.state_dict(), 'models/model_vgg19')

## Test model

In [ ]:
# model = TheModelClass(*args, **kwargs)
# model.load_state_dict(torch.load('models/model_resnet50'))

In [30]:
model.eval()

def test_model(test_loader, model, device=None):
    predictions = []
    
    with torch.no_grad():
        for image, label in test_loader:
            # Move to GPU
            image = image.to(device)
            label = label.to(device)
            
            output = model(image)
            prediction = (output.argmax(1)).cpu().detach().numpy()
            predictions.extend(prediction)
    
    return predictions

In [31]:
predictions = test_model(test_loader, model, device)

In [32]:
predictions = [x + 1 for x in predictions]

In [33]:
rijks_prediction = pd.DataFrame(
    {'filename': test_set_filename,
     'label': predictions
    }
)

rijks_prediction

,filename,label
0,cropped_images/img_1.jpg,2
1,cropped_images/img_2.jpg,7
2,cropped_images/img_3.jpg,2
3,cropped_images/img_4.jpg,3
4,cropped_images/img_5.jpg,7
...,...,...
761,cropped_images/img_762.jpg,3
762,cropped_images/img_763.jpg,3
763,cropped_images/img_764.jpg,4
764,cropped_images/img_765.jpg,7


In [34]:
rijks_prediction.rename(columns = {'label': 'predicted_label'}, inplace=True)
rijks_prediction['actual_label'] = df_test['label']
rijks_prediction

,filename,predicted_label,actual_label
0,cropped_images/img_1.jpg,2,7
1,cropped_images/img_2.jpg,7,7
2,cropped_images/img_3.jpg,2,7
3,cropped_images/img_4.jpg,3,7
4,cropped_images/img_5.jpg,7,1
...,...,...,...
761,cropped_images/img_762.jpg,3,7
762,cropped_images/img_763.jpg,3,7
763,cropped_images/img_764.jpg,4,7
764,cropped_images/img_765.jpg,7,3


In [35]:
correct = (rijks_prediction['predicted_label'] == rijks_prediction['actual_label'])
accuracy = (correct.sum() / correct.size) * 100
accuracy

18.5378590078329

In [37]:
rijks_prediction.to_csv('predictions/vgg19_predictions.csv', index=False)